# In this Notebook, you have 3 section.
# 1. with Efficient Net archtecture --> to learn tranfer learning
*     1. With Transfer Learning I am able to achieve only 90% Accuracy.
*     2. Try to handle imbalanced data using class weight

# 2. Tried to build own model --> I couldn't get all things good. have tryafter efficient net
# 3. Tried to create Encoder + Decoder Archtecture. But result is very bad, have to work on model building 
# Note, code doesn't have submission part yet 

# 1. Using Efficient Net to classifcation

In [ ]:
TF Record Basics :https://www.kaggle.com/ryanholbrook/tfrecords-basics

* 1. All required libraries 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from functools import partial

import numpy as np
from os import path
import os

import tensorflow_addons as tfa
from functools import partial
from kaggle_datasets import KaggleDatasets

In [ ]:
# setting ploting background 
sns.set_style("dark")
TPU = True

* Reading Data & Visualising  images 

*  Reading data set to pandas dataframe, and updating file path 

In [ ]:
train_csv = "../input/seti-breakthrough-listen/train_labels.csv"
train_df_master = pd.read_csv( train_csv)
train_df_master.head()

In [ ]:
train_df_master.shape

* Upading each image path

In [ ]:
train_df_master["path"] = train_df_master["id"].apply( lambda x: "../input/seti-breakthrough-listen/train/"+ str(x[0]) +"/"+x +".npy" )
train_df_master.head()

***** Checking is file path constructed is correct or not by verifying file path using pathexist function

In [ ]:
for x in range( 1, 5 ):    
    file_to_check = train_df_master["path"] [ np.random.randint ( 0, train_df_master.shape[0] ) ]
    
    if( path.exists ( file_to_check ) ):
        print ( "{} file exists...!".format( file_to_check.split("/")[-1] ) )
    else:
        print ( "{} file Does not exists...!".format( file_to_check.split("/")[-1] ) )

***** random file exists which confirms file path constructio correct we can preceeed to other operations

#  Lets visualise random images 

In [ ]:
def create_heatmap(file_to_load,target ):
    data= np.load( file_to_load ).astype( np.float32 )
    fig = plt.figure( figsize = (5,5) , dpi = 80 )
    
    fig.suptitle( file_to_load +"\b target ==" +str (target) )
    
    print ( file_to_load,target  )
    data = np.vstack ( (data[1], data[3], data[5] ))
    print ( "Shape after vertical stack = {}".format ( data.shape))
    #data = np.resize( data, (256,256,3))
    #for i in range( 0, 6 ):
    plt.subplot( 1, 1,1 )
    plt.imshow( data, interpolation='nearest', aspect='auto' )
    
    plt.tight_layout()
    plt.show()

In [ ]:
for i in range(0,4):
    
    if i % 2== 0 :
        
        file_name, target = train_df_master[ ["path","target"] ].iloc[  np.random.randint(train_df_master.shape[0] ) ].values 
        
    else:
        target,file_name = train_df_master[ train_df_master["target"]== 1] [["target","path"]] .iloc[  np.random.randint(train_df_master[ train_df_master["target"]== 1] [["target","path"]].shape[0] ) ].values 
        
    create_heatmap ( file_name, target)    

# * Creating generator for efficient net classification 

In [ ]:
   
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
   
    
except:
    strategy = tf.distribute.get_strategy()
    
print('Number of replicas:', strategy.num_replicas_in_sync)



class CONFIG():
    
    SAMPLE_SIZE= 10000
    IMAGE_WIDTH = 819
    IMAGE_HEIGHT= 256
    BATCH_SIZE = 100 
    RANDOM_STATE = 200
    CHANNEL = 3
    
    

train_df_2 = train_df_master.head( CONFIG.SAMPLE_SIZE )
train_df,test_df  = train_test_split( train_df_2, test_size = 0.2,random_state= CONFIG.RANDOM_STATE, shuffle= True ,stratify= train_df_2["target"] )
valdation_df,test_df = train_test_split( test_df, test_size = 0.1, random_state= CONFIG.RANDOM_STATE, shuffle= True ,stratify= test_df["target"] )

class_weights = compute_class_weight('balanced', 
                                    classes=np.unique(train_df['target'].values),
                                    y=train_df['target'].values)

class_weights_dict = {key: val for key, val in zip(np.unique(train_df['target'].values), class_weights)}
class_weights_dict     

In [ ]:
train_df.shape

# Checking distribution of Target variable between train , test & validation split 

In [ ]:
fig, ax  = plt.subplots( nrows = 1, ncols = 3, figsize = ( 15, 5 ), dpi = 90  )
ax[0].set_title( label = "Train Sample Distribution")
sns.countplot( train_df["target"]  ,ax = ax[0])
ax[1].set_title( label = "Test Sample Distribution")
sns.countplot( test_df["target"]  ,ax = ax[1])
ax[2].set_title( label = "Validation Sample Distribution")
sns.countplot( valdation_df["target"]  ,ax = ax[2])
plt.show()

# Creating Data generator & takingonly axis 0,2,4 from numpy arrya of 6  channels.
# Channel 1,3,5 are signal coming from man made sources

In [ ]:
'''
def Read_numpy_image( file_name_list, traget_list, train = False ):
    
    def gen():
        for each_file, target in zip( file_name_list,traget_list ):
            
            data = np.load( file_name )
            data = np.dstack( (data[0],data[2], data[3]) )
            data = np.resize(data, ( CONFIG.IMAGE_WIDTH,CONFIG.IMAGE_HEIGHT, 3 ) )
        
            yield  data, target 
            
    return ( gen )


def map_func ( x, y  ):
    return ( x, y )



def DATA_SET_GENERATOR ( df ):
    
        AUTOTUNE = tf.data.AUTOTUNE
        train_dataset = tf.data.Dataset.from_generator(generator=Read_numpy_image( file_name_list= df["path"], 
                                                                                   traget_list = df["target"] 
                                                                                 ),
                                                      output_types = ( np.float32,np.float32),
                                                      output_shapes= ( ( CONFIG.IMAGE_WIDTH, CONFIG.IMAGE_HEIGHT,3 ), 1)
                                                      )
        train_dataset.map( map_func=map_func,
                            num_parallel_calls=AUTOTUNE,
                            # Order does not matter.
                            deterministic=False#,
                            #prefetch(AUTO)
                         )
        train_dataset = train_dataset.batch( CONFIG.BATCH_SIZE, drop_remainder=False)
        # Prefetch some batches.
        train_dataset = train_dataset.prefetch(AUTOTUNE)
    
        return ( train_dataset )
'''

In [ ]:

def read_numpy_data( file_name_result ):

    file_name=file_name_result
    data = np.load( file_name.numpy() ).astype( np.float32 )    
    data = np.vstack( ( data[1], data[3], data[5] ) )
    return  np.dstack ( [ data, data, data] ) 



def resize_image( df_dict ):

    [image, ] =  tf.py_function( read_numpy_data,  [ df_dict["path"] ], [tf.float32] )
    image.set_shape( ( CONFIG.IMAGE_HEIGHT,CONFIG.IMAGE_WIDTH, CONFIG.CHANNEL ) )
    image = tf.image.resize( image , ( CONFIG.IMAGE_HEIGHT,CONFIG.IMAGE_WIDTH ) )
    label = df_dict["target"]
    label = tf.cast( label, tf.int16 )

    return  image, label 




def create_tf_dataset( dataframe ):

    tf_dataset = tf.data.Dataset.from_tensor_slices ( dict ( dataframe[ ["path","target"] ] ) ) 
    tf_dataset = ( tf_dataset
                  .shuffle (1024)
                  .map( resize_image,num_parallel_calls= tf.data.AUTOTUNE)
                  .batch( CONFIG.BATCH_SIZE )
                  .prefetch( tf.data.AUTOTUNE)
                 )

    return  tf_dataset  

train_dataset = create_tf_dataset ( train_df )
test_dataset = create_tf_dataset ( test_df )

val_dataset = create_tf_dataset ( valdation_df )




TRAIN_STEPS_PER_EPOCH= int ( train_df.shape[0]/CONFIG.BATCH_SIZE )
VALIDATION_STEPS_PER_EPOCH = int ( valdation_df.shape[0]/CONFIG.BATCH_SIZE )

if ( TRAIN_STEPS_PER_EPOCH * CONFIG.BATCH_SIZE ) !=  train_df.shape[0]: TRAIN_STEPS_PER_EPOCH= TRAIN_STEPS_PER_EPOCH+1
if ( VALIDATION_STEPS_PER_EPOCH * CONFIG.BATCH_SIZE ) !=  valdation_df.shape[0]: VALIDATION_STEPS_PER_EPOCH= VALIDATION_STEPS_PER_EPOCH+1


    

* Creating Tes, Train & validation Tensor data 

In [ ]:
'''
train_dataset = DATA_SET_GENERATOR ( train_df )
test_dataset = DATA_SET_GENERATOR ( test_df )
val_dataset = DATA_SET_GENERATOR ( valdation_df )

TRAIN_STEPS_PER_EPOCH= int ( train_df.shape[0]/CONFIG.BATCH_SIZE )
VALIDATION_STEPS_PER_EPOCH = int ( valdation_df.shape[0]/CONFIG.BATCH_SIZE )
if ( TRAIN_STEPS_PER_EPOCH * CONFIG.BATCH_SIZE ) !=  train_df.shape[0]: TRAIN_STEPS_PER_EPOCH= TRAIN_STEPS_PER_EPOCH+1
if ( VALIDATION_STEPS_PER_EPOCH * CONFIG.BATCH_SIZE ) !=  valdation_df.shape[0]: VALIDATION_STEPS_PER_EPOCH= VALIDATION_STEPS_PER_EPOCH+1
'''

# UseEfficient net to start classification 

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, lr_min=0.000001, lr_rampup_epochs=20, lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync
    
    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay ** (epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

In [ ]:
def create_train_val_loss_plot( history):
    
    key = list ( history.history.keys() )
    
    key = [ key[1],key[3] ]
    sns.lineplot (x = range(0, len(history.history[key[0]] )), y = history.history[key[0]] ) 
    sns.lineplot (x = range(0, len(history.history[key[1]] )), y = history.history[key[1]] ) 
    plt.legend( key, loc ="upper right")
    plt.xlabel("Epoch")
    plt.ylabel(key[0])
    plt.show()

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0001)

new_eff_model.compile( optimizer=optimizer, loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
lrfn = build_lrfn()

model_history = new_eff_model.fit_generator(train_dataset,
                                             class_weight= class_weights_dict,
                                             steps_per_epoch= TRAIN_STEPS_PER_EPOCH, 
                                             epochs = 10, 
                                             validation_data= val_dataset,
                                             validation_steps = VALIDATION_STEPS_PER_EPOCH,
                                            callbacks=[ tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1),
                                                      tf.keras.callbacks.ModelCheckpoint( os.path.join("./model.h5"), 
                                                                                         monitor='train_loss', 
                                                                                         verbose=0,
                                                                                         save_best_only=True, 
                                                                                         save_weights_only=False,
                                                                                         mode='auto', 
                                                                                         save_freq='epoch'
                                                                                        )
                                                      ]
                                        )

In [ ]:
efff_net.summary()

In [ ]:
create_train_val_loss_plot ( model_history )

In [ ]:
horizontal_flip = tf.keras.layers.experimental.preprocessing.RandomFlip( "horizontal_and_vertical", seed=CONFIG.RANDOM_STATE )
random_rotation = tf.keras.layers.experimental.preprocessing.RandomRotation ( 0.2 )
contrast = tf.keras.layers.experimental.preprocessing.RandomContrast( factor = 0.2,  seed=CONFIG.RANDOM_STATE  )
noise = tf.keras.layers.GaussianNoise( stddev= 0.2 )

In [ ]:

input_mo = tf.keras.layers.Input(shape= ( CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH, CONFIG.CHANNEL) )

efff_net =tf.keras.applications.EfficientNetB7(include_top = False, 
                                               weights ="imagenet" , 
                                               input_shape = ( CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH, CONFIG.CHANNEL) ,
                                               input_tensor = input_mo,
                                               classes=2
                                              )

for i ,layer in enumerate ( efff_net.layers ) :
    if i> 700:
        layer.trainable = False
    else:
        layer.trainable = False
        #horizontal_flip,random_rotation,contrast,
  
model = tf.keras.Sequential( [horizontal_flip,random_rotation,contrast,noise,efff_net,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation= 'tanh'), 
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(200, activation= 'tanh'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100, activation= 'tanh'),
        tf.keras.layers.Dense(1, activation='tanh')
        ])    
        
        
#efff_net( inputs = input_mo)
#eff_net_output = tf.keras.layers.Flatten()( efff_net.layers[-1].output )


#x = layers.GlobalAveragePooling2D()( efff_net.layers[-1].output )
#x = layers.Dropout(0.5)(x)
    
#outputs = layers.Dense(1 )(x)
#outputs = layers.Activation('sigmoid', dtype='float32', name='predictions')(outputs)

#layer_1 = tf.keras.layers.Dense( units =500,activation = "relu" , use_bias = True ) (eff_net_output) 
#layer_2 = tf.keras.layers.Dropout( rate= 0.05 )  ( layer_1 )
#layer_3 =  tf.keras.layers.Dense( units =100,activation = "relu" , use_bias = True )  ( layer_2 )
#layer_4 =  tf.keras.layers.Dropout( rate= 0.05 )  ( layer_3 )
#layer_5 =  tf.keras.layers.Dense( units =1,activation = "sigmoid" , use_bias = True ) ( layer_4 ) 
#layer_6 =  tf.keras.layers.Dropout( rate= 0.25 )  ( layer_5 )
#layer_7 =  tf.keras.layers.Dense( units =1,activation = "sigmoid" , use_bias = True )  ( layer_6 )

#new_eff_model_2 = models.Model( inputs = input_mo, outputs = outputs )

optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0001)


In [ ]:
model.compile( optimizer= "adam", #optimizer
                        loss="binary_crossentropy",
                        metrics=[tf.keras.metrics.AUC(curve='ROC')])


In [ ]:


#import neural_structured_learning as nsl

lrfn = build_lrfn()

# Wrap the model with adversarial regularization.
#adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
#adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)

#adv_model.compile( optimizer= "adam", #optimizer
#                        loss="binary_crossentropy",
#                        metrics=[tf.keras.metrics.AUC(curve='ROC')])


model_history2 = model.fit_generator(train_dataset,
                                              class_weight= class_weights_dict,
                                             steps_per_epoch= TRAIN_STEPS_PER_EPOCH, 
                                             epochs =10, 
                                             validation_data= val_dataset,
                                             validation_steps = VALIDATION_STEPS_PER_EPOCH,
                                            callbacks=[ tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1),
                                  tf.keras.callbacks.ModelCheckpoint( "./model2.h5", 
                                                                     monitor='train_loss', 
                                                                     verbose=0,
                                                                     save_best_only=True, 
                                                                     save_weights_only=True,
                                                                     mode='auto', 
                                                                     save_freq='epoch'
                                                                    )
                                  ]
                        )

In [ ]:
create_train_val_loss_plot ( model_history2 )

In [ ]:
 predict = model.predict( test_dataset ) 

In [ ]:
def confusion_matrix ( model, test_data, ):
    
    predicted_data = model.predict( test_data )

In [ ]:
## Thanks to UPGRAD, from one othere project I got this code, hoping it will work for Auto encoder architecture neural network 

def data_generator_Rev_01( file_list , image_width,image_length, num_channel):
    
    def generator():
        

            #batch_data = np.zeros( (  image_width, image_length, num_channel ) )
            #batch_file_list = np.random.permutation ( file_list )

            #for count , each_file in enumerate( batch_file_list) :
            for each_file in file_list:
                
                np_data = np.load( each_file ).astype( np.float32 )    
                np_data = np.dstack( ( np_data[0], np_data[2], np_data[4] ) ) #( np_data[0], np_data[2], np_data[4] )

                np_data = np.resize( np_data,( image_width,image_length, num_channel ) )

                yield np_data,np_data
            
    return ( generator )



def map_func( x ):
    return x

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
#file_list = train_df[train_df["target"] == 0]["path"].values()
gen = data_generator_Rev_01 ( file_list = file_list,
                                #batch_size = CONFIG["BATCH_SIZE"],
                                image_width = CONFIG["image_width"], 
                                image_length = CONFIG["image_height"], 
                                num_channel = CONFIG["num_channel"]
                               )
dataset = tf.data.Dataset.from_generator(
    generator=gen, 
    output_types=(np.float32, np.int32), 
    output_shapes=( ( CONFIG["image_width"],CONFIG["image_height"],3 ), ( CONFIG["image_width"],CONFIG["image_height"],3 ) )
)

# Parallelize the augmentation.
dataset = dataset.map(
    map_func, 
    num_parallel_calls=AUTOTUNE,
    # Order does not matter.
    deterministic=False#,
    #prefetch(AUTO)
)
dataset = dataset.batch( CONFIG["BATCH_SIZE"], drop_remainder=False)
# Prefetch some batches.
dataset = dataset.prefetch(AUTOTUNE)

# Data is already normailsed to have mean of 0, not doing any preprocessing on the each images.
# Will start wriing generator function for Encoders. Because of below reason
* Avalable generator wont work for Autoencoder, tried few they didn't work. Found one code f using TFrecod, it was to heay for me.
* So i took one of generator code I got from Upgrad assignment,

* Each numpy file has 6 channels, not all channels are use full found few discussion thread mentioning using only 3 channels.
* I'll be exploring with 3 channels first, later I can expand to 6 channels
* discussion therad : 
* https://www.kaggle.com/c/seti-breakthrough-listen/discussion/239173
* https://www.kaggle.com/c/seti-breakthrough-listen/discussion/238241

channels ABACAD. select only cannel B,C &D

# Got this facinating link to for data generator.
https://stackoverflow.com/questions/64356769/tensorflow2-x-custom-data-generator-with-multiprocessing

my method was taking 160s, were as method in the above link takes 37s. close 6 time faster wow.

Function implemented with above method is with "data_generator_Rev_01"

In [ ]:
## Thanks to UPGRAD, from one othere project I got this code, hoping it will work for Auto encoder architecture neural network 

def data_generator( file_list, batch_size,image_width, image_length, num_channel = 3 ):
    
    
    number_of_batches = int ( len ( file_list )/ batch_size )
    
    leftoved_file_count = 0
    
    if (number_of_batches * batch_size) == len( file_list ):
        leftover= False
    else:
        leftover = True
        leftoved_file_count = len ( file_list ) -  (number_of_batches * batch_size)
    
    
    while True:
        
        for each_batch in range( 0, number_of_batches ):
            
            if  leftover & ( each_batch == number_of_batches -1 )  :
                
                batch_data = np.zeros( ( leftoved_file_count, image_width, image_length, num_channel ) ) # [ Batch number]
                batch_file_list = np.random.permutation ( file_list[ : - leftoved_file_count ] )
                
                
            else:
                
                batch_data = np.zeros( ( batch_size, image_width, image_length, num_channel ) )
                batch_file_list = np.random.permutation ( file_list[ ( each_batch * batch_size ) : (batch_size * (each_batch+ 1 ) ) ] )
            
            for count , each_file in enumerate( batch_file_list) :
                
                np_data = np.load( each_file ).astype( np.float32 )    
                np_data = np.dstack( ( np_data[0], np_data[2], np_data[4] ) )
                
                batch_data[count] = resize( np_data,( image_width,image_length, num_channel ) )
        
        yield batch_data

In [ ]:
CONFIG = { "BATCH_SIZE" : 50, "image_width": 224, "image_height": 224,"num_channel" : 3, "sample_size" : 1000, }

In [ ]:
train_data_x = data_generator ( file_list = np.random.permutation (train_df["path"]) [: CONFIG["sample_size"]],
                                batch_size = CONFIG["BATCH_SIZE"],
                                image_width = CONFIG["image_width"], 
                                image_length = CONFIG["image_height"], 
                                num_channel = CONFIG["num_channel"]
                               )

if CONFIG["sample_size"] % CONFIG["BATCH_SIZE"] == 0 :
    num_steps = int( CONFIG["sample_size"]/CONFIG["BATCH_SIZE"]) 
else:
    num_steps = int( CONFIG["sample_size"] /CONFIG["BATCH_SIZE"])  + 1

In [ ]:
#next ( train_data_x )

In [ ]:
model_input  =   ( CONFIG["image_height"], CONFIG["image_width"], CONFIG["num_channel"])
encoder = models.Sequential()
encoder.add( layers.Conv2D ( filters = 224 , kernel_size = 2, padding ="same",strides = 1, activation = "tanh",use_bias = True, input_shape = model_input  ) )
#encoder.add( layers.BatchNormalization ( ) )
encoder.add( layers.Conv2D ( filters = 128 , kernel_size = 3, padding ="same",strides = 2,activation = "tanh",use_bias = True ) )#, name="layer_02" )
#encoder.add( layers.BatchNormalization ( ) )
encoder.add( layers.Conv2D ( filters = 64 , kernel_size = 4, padding ="same",strides = 2,activation = "tanh",use_bias = True ) )#, name="layer_03")
#encoder.add( layers.BatchNormalization ( ) si
encoder.add( layers.Conv2D ( filters = 32 , kernel_size = 4, padding ="same",strides = 2,activation = "tanh",use_bias = True ) )#, name="layer_04")
#encoder.add( layers.BatchNormalization ( ) )
encoder.add( layers.Conv2D ( filters = 16 , kernel_size = 4, padding ="same",strides = 2,activation = "tanh",use_bias = True ) )#, name="layer_04")
encoder.add( layers.Conv2D ( filters = 8 , kernel_size = 4, padding ="same",strides = 2,activation = "tanh",use_bias = True ) )#, name="layer_04")
#decoder = models.Sequential()
encoder.add( layers.Conv2DTranspose ( filters = 8 , kernel_size =4 , padding ="same",strides = 2,activation = "tanh",use_bias = True ) )#, name="layer_04")                    
#encoder.add( layers.BatchNormalization ( ) )
encoder.add( layers.Conv2DTranspose ( filters = 16 , kernel_size = 4, padding ="same",strides = 2,activation = "tanh",use_bias = True ) )#, name="layer_03")
#encoder.add( layers.BatchNormalization ( ) )
encoder.add( layers.Conv2DTranspose ( filters = 32 , kernel_size = 4, padding ="same",strides = 2,activation = "tanh",use_bias = True ) )#, name="layer_02" )
#encoder.add( layers.BatchNormalization ( ) )
encoder.add( layers.Conv2DTranspose ( filters = 64 , kernel_size = 4, padding ="same",strides = 2,activation = "tanh",use_bias = True) )
#encoder.add( layers.BatchNormalization ( ) )
encoder.add( layers.Conv2DTranspose ( filters = 128 , kernel_size =3, padding ="same",strides = 2,activation = "tanh",use_bias = True) )
encoder.add( layers.Conv2DTranspose ( filters = 3 , kernel_size = 2, padding ="same",strides = 1,activation = "tanh",use_bias = True) )


encoder.compile ( optimizer= "adam", 
               loss="mse"
               )

encoder.summary()

In [ ]:
#encoder.fit( train_data_x,train_data_x,epochs = 10  ,steps_per_epoch = num_steps ,verbose=1)

In [ ]:
## Thanks to UPGRAD, from one othere project I got this code, hoping it will work for Auto encoder architecture neural network 

def data_generator_Rev_01( file_list , image_width,image_length, num_channel):
    
    def generator():
        

            #batch_data = np.zeros( (  image_width, image_length, num_channel ) )
            #batch_file_list = np.random.permutation ( file_list )

            #for count , each_file in enumerate( batch_file_list) :
            for each_file in file_list:
                
                np_data = np.load( each_file ).astype( np.float32 )    
                np_data = np.dstack( ( np_data[0], np_data[2], np_data[4] ) ) #( np_data[0], np_data[2], np_data[4] )

                np_data = np.resize( np_data,( image_width,image_length, num_channel ) )

                yield np_data,np_data
            
    return ( generator )



def map_func( x, y):
    return x, y

In [ ]:
file_list = train_df[train_df["target"] == 0]["path"].values [: CONFIG["sample_size"]]

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
#file_list = train_df[train_df["target"] == 0]["path"].values()
gen = data_generator_Rev_01 ( file_list = file_list,
                                #batch_size = CONFIG["BATCH_SIZE"],
                                image_width = CONFIG["image_width"], 
                                image_length = CONFIG["image_height"], 
                                num_channel = CONFIG["num_channel"]
                               )
dataset = tf.data.Dataset.from_generator(
    generator=gen, 
    output_types=(np.float32, np.int32), 
    output_shapes=( ( CONFIG["image_width"],CONFIG["image_height"],3 ), ( CONFIG["image_width"],CONFIG["image_height"],3 ) )
)

# Parallelize the augmentation.
dataset = dataset.map(
    map_func, 
    num_parallel_calls=AUTOTUNE,
    # Order does not matter.
    deterministic=False#,
    #prefetch(AUTO)
)
dataset = dataset.batch( CONFIG["BATCH_SIZE"], drop_remainder=False)
# Prefetch some batches.
dataset = dataset.prefetch(AUTOTUNE)

In [ ]:
lrfn = build_lrfn()
Auto_Encoder_History = encoder.fit_generator( dataset ,
                                             epochs = 30 ,
                                             callbacks=[ tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)] )

In [ ]:
# Reference :https://www.kaggle.com/akhileshdkapse/seti-cnn-encoder-decoder-model-tpu
#https://www.kaggle.com/awsaf49/seti-bl-tf-starter-tpu
#https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/

In [ ]:
loss_value = Auto_Encoder_History.history["loss"]
sns.lineplot( y = loss_value ,x = range( 0, len(loss_value) )  )

In [ ]:
def data_generator_for_predictor( file_list,image_width, image_length, num_channel = 3 ):
    
        #batch_data = np.zeros( (  image_width, image_length, num_channel ) )

        np_data = np.load( file_list ).astype( np.float32 )    
        np_data = np.dstack( ( np_data[1], np_data[3], np_data[5] ) )

        batch_data = np.resize( np_data,( 1 ,image_width,image_length, num_channel ) )

        return ( batch_data )

In [ ]:
file_path_name = train_df ["path"].iloc [ np.random.randint( 1000) ]
file_path_name = train_df[train_df["target"] == 0 ] ["path"].iloc [10  ]

d = data_generator_for_predictor( file_path_name,
                                image_width = CONFIG["image_width"], 
                                image_length = CONFIG["image_height"], 
                                num_channel = CONFIG["num_channel"]
                                )
decode_data = encoder.predict( d )
mean_absoulte_error = tf.keras.losses.MeanAbsoluteError()
mean_absoulte_error( decode_data[0] , d[ 0 ] )

In [ ]:
len( list ( train_df["path"][:100] ) )

In [ ]:
mean_absoulte_error = tf.keras.losses.MeanAbsoluteError()
loss_value = []
train_df_2 = train_df.head( 100 )
for each_file in train_df["path"][:100]:
    d = data_generator_for_predictor( each_file,
                                image_width = CONFIG["image_width"], 
                                image_length = CONFIG["image_height"], 
                                num_channel = CONFIG["num_channel"]
                                )
    decode_data = encoder.predict( d )

    loss_value.append( mean_absoulte_error( decode_data[0] , d[ 0 ] ).numpy() )

train_df_2["MAE"] = loss_value

In [ ]:
train_df_2.head()

In [ ]:
fig = plt.figure(figsize=( 10,10))
sns.lineplot (data = train_df_2, y="MAE",x ="id" , hue = "target" )
plt.show()

In [ ]:
train_df_2[ ( train_df_2["MAE"]>0.6) & (train_df_2["taget"] == 1 ) ]

In [ ]:
target = train_df [ train_df["path"] ==file_path_name ]["target"].values[0]
create_heatmap ( file_path_name, target)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "../input/seti-breakthrough-listen/train/",
  validation_split=0.2,
  subset="training",
  seed=123,
  #image_size=(img_height, img_width),
  batch_size=30)

In [ ]:
a.shape

In [ ]:
a[1] = np.zeros( (2,2,3))

In [ ]:
data_path = "../input/seti-breakthrough-listen/train_labels.csv"
df = pd.read_csv( data_path, sep = ",")

In [ ]:
df.head()

In [ ]:
print ("number of signal with target 1 = {}".format( df[ df["target"] == 1 ].shape) )
print ("number of signal with target 0 = {}".format( df[ df["target"] == 0 ].shape) )

In [ ]:
sns.set_style("darkgrid")

In [ ]:
fig = plt.figure( figsize = ( 8,10) , dpi =80 )
ax = fig.add_subplot ( 111)
plt.title( " Count of Anomally & non Anamolly data ")
sns.countplot( df["target"], ax = ax  )
ax.xaxis.set_tick_params( labelsize = 20 )
ax.yaxis.set_tick_params( labelsize = 20 )
ax.yaxis.label.set_fontsize( 20 )
ax.xaxis.label.set_fontsize( 20 )

In [ ]:
df.tail()

In [ ]:
train_path ="../input/seti-breakthrough-listen/train/"
df["path"]= df["id"].apply( lambda x: train_path+x[0]+"/"+ x +".npy" )

In [ ]:
df.head()

In [ ]:
np.load( df["path"].iloc[ np.random.randint(df.shape[0] )] ).shape

In [ ]:
def create_heatmap():
    data= np.load( df["path"].iloc[ np.random.randint(df.shape[0] )] )
    fig, axis = plt.subplots( nrows = 1, ncols = 6 , dpi = 80 , figsize  =(20,5))
    axis = axis.flatten()
    for i,each_axis in enumerate( axis) :    sns.heatmap( data[i], ax = each_axis )

In [ ]:
create_heatmap()
create_heatmap()
create_heatmap()

In [ ]:
## IMPORTING REQUIRED TENSOR FLOW 

import tensorflow as tf
import tensorflow.keras as keras
from  tensorflow.keras import layers,models
from sklearn.model_selection import train_test_split
import datetime
import os

In [ ]:
CONFIG = {"image_height" : 224,
          "image_width"  : 224,
          "batch_size" :100,
          "random_state" :  100
         }

In [ ]:
test_df,train_df  = train_test_split ( train_df[ ["path","target" ] ], 
                             test_size= 0.8, 
                             random_state= CONFIG["random_state"], 
                             shuffle =True )

In [ ]:
def creat_plot(path):
    data= np.load( path )
    fig, axis = plt.subplots( nrows = 1, ncols = 6 , dpi = 80 , figsize  =(20,5))
    axis = axis.flatten()
    for i,each_axis in enumerate( axis) :    sns.heatmap( data[i], ax = each_axis )

* Visualizing the data for random samples after resizingdata

In [ ]:
creat_plot(train_df["path"].iloc [10] )
creat_plot(train_df["path"].iloc[20] )

In [ ]:
print ( "Number of file for training = {}".format( train_df.shape ))
print ( "Number of file has anamoly  = {}".format( test_df.shape ))

In [ ]:
fig = plt.figure( figsize = ( 15,8) , dpi =80 )
ax = fig.add_subplot ( 121)
plt.title( " Count of Anomally & non Anamolly data in train_data  ")
sns.countplot( train_df["target"], ax = ax  )
ax.xaxis.set_tick_params( labelsize = 20 )
ax.yaxis.set_tick_params( labelsize = 20 )
ax.yaxis.label.set_fontsize( 20 )
ax.xaxis.label.set_fontsize( 20 )

ax = fig.add_subplot ( 122)
plt.title( " Count of Anomally & non Anamolly data in test data  ")
sns.countplot( test_df["target"], ax = ax  )
ax.xaxis.set_tick_params( labelsize = 20 )
ax.yaxis.set_tick_params( labelsize = 20 )
ax.yaxis.label.set_fontsize( 20 )
ax.xaxis.label.set_fontsize( 20 )

In [ ]:
def read_numpy_data( file_name_result ):
    
    file_name=file_name_result
    data = np.load( file_name.numpy() ).astype( np.float32 )    
    data = np.dstack( ( data[0], data[2], data[4] ) )
    return ( data )
    

In [ ]:
def resize_image( df_dict ):
    
    [image, ] =  tf.py_function( read_numpy_data,  [ df_dict["path"] ], [tf.float32] )
    image.set_shape(( 273, 256, 3 ) )
    image = tf.image.resize( image , ( CONFIG["image_height"],  CONFIG["image_width"] ) )
    label =  df_dict["target"]#.reshape((-1,1))
    label = tf.cast( label, tf.float32 ), (-1,1) 
  
    return ( image, label )

In [ ]:
def create_tf_dataset( dataframe ):
    
    tf_dataset = tf.data.Dataset.from_tensor_slices ( dict ( dataframe[ ["path","target"] ] ) ) 
    tf_dataset = ( tf_dataset
                  .shuffle (1024)
                  .map( resize_image,num_parallel_calls= tf.data.AUTOTUNE)
                  .batch( CONFIG["batch_size"] )
                  .prefetch(tf.data.AUTOTUNE)
                 )
    
    return ( tf_dataset )

In [ ]:
train_tf_dataset = create_tf_dataset ( train_df )
test_tf_dataset = create_tf_dataset ( test_df )

In [ ]:
CONFIG["model_name"] = "model_1"
CONFIG["group"] = "model_1"
CONFIG["run_name"] = "model_1_baseline"

In [ ]:
DEFAULT_MODEL = False
if DEFAULT_MODEL: ## bulding model 
    model_input  = (  CONFIG["image_height"], CONFIG["image_width"], 3 )
    model = models.Sequential()
    model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 3,3), padding ="valid",activation = "relu",use_bias = True, input_shape = model_input  ) )
    model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 4,4), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_02" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 5,5), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_03")
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 6,6), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_04")
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 7,7), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_04")
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 8,8), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_04")
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 9,9), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 12,12), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 15,15), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 15,15), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 20,20), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 25,25), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 20,20), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 25,25), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    #model.add( layers.Conv2D ( filters = 32 , kernel_size = ( 25,25), padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_05" )
    model.add( layers.Flatten () )
    model.add( layers.Dense  ( units = 32 , use_bias = True, activation = "relu"))#,name ="dense_01")
    model.add( layers.Dense  ( units = 16 , use_bias = True, activation = "sigmoid") )#,name ="dense_02" )
    model.add( layers.Dense  ( units = 8 , use_bias = True, activation = "sigmoid") )#,name ="dense_03" )
    model.add( layers.Dense  ( units = 1 , use_bias = True, activation = "sigmoid") )#,name ="dense_04" )



    model.compile ( optimizer= "adam", 
                   loss= "binary_crossentropy",
                   metrics=[tf.keras.metrics.AUC(curve='ROC')])

    model.summary()

In [ ]:
'''
## initilisize WandB
if DEFAULT_MODEL: ## bulding model 
    wandb_store = wandb.init( project = "SETI_Breakthrough_Listening",
                            config = CONFIG,
                            group =CONFIG["group"],
                            job_type="train",
                            name=CONFIG["run_name"]
                            )
'''

In [ ]:
'''
if DEFAULT_MODEL: ## bulding model 
    model.fit( train_tf_dataset ,epochs = 10 , callbacks= [ WandbCallback(log_weights=True) ])

    # Saving model

    model_path = "./WandB/"+CONFIG["model_name"]
    os.makedirs(model_path,exist_ok = True )
    model_count = len ( os.listdir( model_path )) +1
    model.save(model_path +"model_count" +".h5" )

    wandb_store.finish()
'''

In [ ]:
CONFIG["model_name"] = "encoder_decoder"
CONFIG["group"] = "encoder_decoder"
CONFIG["run_name"] = "encoder_decore_structure"

In [ ]:
class Auto_Encoder (tf.keras.Model ):
    
    def __init__(self, image_height , image_width, dimension ):
        
        super( Auto_Encoder, self ).__init__( )
        self.model_input  = (  image_height, image_width, dimension )
                             
        self.encoder_model=  self.encoder_func()
        
        #self.encoder_out_put_shape = self.encoder_model.output_shape
        self.decoder_model = self.decoder_func()
    
    def encoder_func(self ):
        
        model_input  = self.model_input #(  CONFIG["image_height"], CONFIG["image_width"], 3 )
        encoder = models.Sequential()
        encoder.add( layers.Conv2D ( filters = 128 , kernel_size = 2, padding ="same",strides = 2, activation = "relu",use_bias = True, input_shape = model_input  ) )
        encoder.add( layers.Conv2D ( filters = 64 , kernel_size = 2, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_02" )
        encoder.add( layers.Conv2D ( filters = 32 , kernel_size = 2, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_03")
        encoder.add( layers.Conv2D ( filters = 16 , kernel_size = 2, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_04")
        encoder.add( layers.Conv2D ( filters = 8 , kernel_size = 2, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_04")

        return ( encoder )
                             
    def decoder_func ( self):
         
        decoder = models.Sequential() 
        decoder.add( layers.Conv2DTranspose ( filters = 8 , kernel_size = 2 , padding ="valid",activation = "relu",use_bias = True, input_shape = self.encoder_model.output_shape[1:]  ) )#, name="layer_04")                    
        decoder.add( layers.Conv2DTranspose ( filters = 16 , kernel_size = 2, padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_03")
        decoder.add( layers.Conv2DTranspose ( filters = 32 , kernel_size = 2, padding ="valid",activation = "relu",use_bias = True ) )#, name="layer_02" )
        decoder.add( layers.Conv2DTranspose ( filters = 64 , kernel_size = 2, padding ="valid",activation = "relu",use_bias = True) )
        decoder.add( layers.Conv2DTranspose ( filters = 128 , kernel_size =2, padding ="valid",activation = "relu",use_bias = True) )
        decoder.add( layers.Conv2DTranspose ( filters = 3 , kernel_size = 223, padding ="valid",activation = "relu",use_bias = True) )
        
        return( decoder )
    
    def call(self, x):
        encoded = self.encoder_model(x)
        decoded = self.decoder_model(encoded)
        return decoded

In [ ]:
model_input  =   ( CONFIG["image_height"], CONFIG["image_width"], 6) 
encoder = models.Sequential()
encoder.add( layers.Conv2D ( filters = 128 , kernel_size = 2, padding ="same",strides = 1, activation = "relu",use_bias = True, input_shape = model_input  ) )
encoder.add( layers.Conv2D ( filters = 64 , kernel_size = 3, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_02" )
encoder.add( layers.Conv2D ( filters = 32 , kernel_size = 4, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_03")
encoder.add( layers.Conv2D ( filters = 16 , kernel_size = 5, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_04")
encoder.add( layers.Conv2D ( filters = 8 , kernel_size = 6, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_04")

#decoder = models.Sequential()
encoder.add( layers.Conv2DTranspose ( filters = 8 , kernel_size = 6 , padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_04")                    
encoder.add( layers.Conv2DTranspose ( filters = 16 , kernel_size = 5, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_03")
encoder.add( layers.Conv2DTranspose ( filters = 32 , kernel_size = 4, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_02" )
encoder.add( layers.Conv2DTranspose ( filters = 64 , kernel_size = 3, padding ="same",strides = 2,activation = "relu",use_bias = True) )
encoder.add( layers.Conv2DTranspose ( filters = 128 , kernel_size =2, padding ="same",strides = 1,activation = "relu",use_bias = True) )
encoder.add( layers.Conv2DTranspose ( filters = 6 , kernel_size = 2, padding ="same",strides = 1,activation = "relu",use_bias = True) )
        

encoder.compile ( optimizer= "adam", 
               loss= "binary_crossentropy",
               metrics=[tf.keras.metrics.AUC(curve='ROC')])

encoder.summary()

In [ ]:
'''
model_input  =   ( CONFIG["image_height"], CONFIG["image_width"], 3 )
encoder = models.Sequential()
encoder.add( layers.Conv2D ( filters = 128 , kernel_size = 2, padding ="same",strides = 1, activation = "relu",use_bias = True, input_shape = model_input  ) )
encoder.add( layers.Conv2D ( filters = 64 , kernel_size = 3, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_02" )
encoder.add( layers.Conv2D ( filters = 32 , kernel_size = 4, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_03")
encoder.add( layers.Conv2D ( filters = 16 , kernel_size = 5, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_04")
encoder.add( layers.Conv2D ( filters = 8 , kernel_size = 6, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_04")

#decoder = models.Sequential()
encoder.add( layers.Conv2DTranspose ( filters = 8 , kernel_size = 6 , padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_04")                    
encoder.add( layers.Conv2DTranspose ( filters = 16 , kernel_size = 5, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_03")
encoder.add( layers.Conv2DTranspose ( filters = 32 , kernel_size = 4, padding ="same",strides = 2,activation = "relu",use_bias = True ) )#, name="layer_02" )
encoder.add( layers.Conv2DTranspose ( filters = 64 , kernel_size = 3, padding ="same",strides = 2,activation = "relu",use_bias = True) )
encoder.add( layers.Conv2DTranspose ( filters = 128 , kernel_size =2, padding ="same",strides = 1,activation = "relu",use_bias = True) )
encoder.add( layers.Conv2DTranspose ( filters = 3 , kernel_size = 10, padding ="same",strides = 1,activation = "relu",use_bias = True) )
        

encoder.compile ( optimizer= "adam", 
               loss= "binary_crossentropy",
               metrics=[tf.keras.metrics.AUC(curve='ROC')])

encoder.summary()
'''

In [ ]:
CONFIG["batch_size"] = 50
def resize_image( df_dict ):
    
    [image, ] =  tf.py_function( read_numpy_data,  [ df_dict["path"] ], [tf.float32] )
    image.set_shape(( 273, 256, 3 ) )
    image = tf.image.resize( image , ( CONFIG["image_height"],  CONFIG["image_width"] ) )
    label =  df_dict["target"]#.reshape((-1,1))
    label = tf.cast( label, tf.float32 ), (-1,1) 
  
    return  (image,image)
            
def create_tf_dataset( dataframe ):
    
    tf_dataset = tf.data.Dataset.from_tensor_slices ( dict ( dataframe[ ["path","target"] ] ) ) 
    tf_dataset = ( tf_dataset
                  .shuffle (1024)
                  .map( resize_image,num_parallel_calls= tf.data.AUTOTUNE)
                  .batch( CONFIG["batch_size"] )
                  .prefetch(tf.data.AUTOTUNE)
                  
                 )
    
    return ( tf_dataset )

            
train_tf_dataset = create_tf_dataset ( train_df.head(200) )
test_tf_dataset = create_tf_dataset ( test_df.head(50) )

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
train_generator1 = train_datagen.flow_from_directory(
        "../input/seti-breakthrough-listen/train/",
        target_size=(224, 224),
        batch_size=32,
        class_mode=None,
        shuffle=True)

In [ ]:
'''

CONFIG["model_name"] = "encoder_decoder"
CONFIG["group"] = "encoder_decoder"
CONFIG["run_name"] = "encoder_decore_structure"

wandb_store = wandb.init( project = "SETI_Breakthrough_Listening",
                            config = CONFIG,
                            group =CONFIG["group"],
                            job_type="train",
                            name=CONFIG["run_name"]
                            )

encoder.fit( train_generator1,train_generator1 ,epochs = 10 , callbacks= [ WandbCallback(log_weights=True) ])

# Saving model

model_path = "./WandB/"+CONFIG["model_name"]
os.makedirs(model_path,exist_ok = True )
model_count = len ( os.listdir( model_path )) +1
model.save(model_path +"model_count" +".h5" )

wandb_store.finish()
'''

In [ ]:
from tensorflow.keras import utils
#import numpy as np

In [ ]:
class DataGenerator ( utils.Sequence) :
    
    def __init__ (self, data_list, batch_size, #required_channels = [1, 3, 5 ] , 
                  image_height = 224, image_width = 224, 
                  shuffle = True,num_channel = 3  ):
        
        self.data_list = data_list
        self.batch_size = batch_size
        #self.channels_required = required_channels
        self.image_width = image_width
        self.image_height = image_height
        self.shuffle = True 
        self.num_channel =num_channel 
        self.is_there_left_over = False # this to check is there any left images after creating batch slices 
        
        self.number_of_batches = 0
        
        
    def on_epoch_end(self):
        self.indices = np.arrange( len(self.data_list ))
        
        if self.shuffle :
            np.random.shuffle( self.indices )
    
    def __len__(self ):
        
        number_of_batch =int ( len( self.data_list)/ self.batch_size )
        
        ''' below if loop to check for remaining data after slicing with respectto batch'''
        if number_of_batch * self.batch_size !=len( self.data_list):
            number_of_batch = number_of_batch +1
            self.is_there_left_over = True
        
        self.number_of_batches = number_of_batch
        
        return ( number_of_batch  )
    
    def __getitem__(self, idx = 0  ):
        
        x = np.empty( ( self.batch_size, self.image, self.num_channel ) )
        
        if self.is_there_left_over & (idx == (self.number_of_batches - 1 ) ) :
            
            indices = self.indices [ idx * self.batch_size :  ]
            
        else:
            
            indices = self.indices [ idx * self.batch_size : ( idx +1 )* self.batch_size ]
        
        for i, path in enumerate ( indices):
            x[i] = self.read_image( path )
        
        return x 
            
    def read_image ( self, path ):
        
        file_name=file_name_result
        data = np.load( path ).astype( np.float32 )    
        data = np.dstack( ( data[0], data[2], data[4] ) )
        
        return ( data )

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def read_stack_image (  path ):
        
        file_name=file_name_result
        data = np.load( path ).astype( np.float32 )    
        data = np.dstack( ( data[0], data[2], data[4] ) )
        
        return ( data )

In [ ]:
data_gen = ImageDataGenerator( preprocessing_function =  read_stack_image )
train_x = data_gen.flow_from_directory ( "../input/seti-breakthrough-listen/train/", target_size = ( 224,224), 
                                         batch_size = 20, #color_mode ="rgb",
                                        shuffle = True, classes =["test"],
                                       )

In [ ]:
train_x.next()

In [ ]:
train_x = DataGenerator( data_list = train_df["path"], batch_size = 20 )
test_x = DataGenerator( data_list = test_df["path"], batch_size = 20 )

In [ ]:
'''

CONFIG["model_name"] = "encoder_decoder"
CONFIG["group"] = "encoder_decoder"
CONFIG["run_name"] = "encoder_decore_structure"

wandb_store = wandb.init( project = "SETI_Breakthrough_Listening",
                            config = CONFIG,
                            group =CONFIG["group"],
                            job_type="train",
                            name=CONFIG["run_name"]
                            )

encoder.fit( train_x ,epochs = 10 , callbacks= [ WandbCallback(log_weights=True) ])

# Saving model

model_path = "./WandB/"+CONFIG["model_name"]
os.makedirs(model_path,exist_ok = True )
model_count = len ( os.listdir( model_path )) +1
model.save(model_path +"model_count" +".h5" )

wandb_store.finish()
'''

In [ ]:
#train_df["path"]
import tensorflow as tf

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.TFRecordDataset ( list (train_df["path"] ), num_parallel_reads= AUTO )

In [ ]:
list ( train_df["path"] )[:10]

In [ ]:
def read_stack_image (  path ):
        
        #file_name=file_name_result
        #data = np.load( path ).astype( np.float32 )    
        #data = np.dstack( ( data[0], data[2], data[4] ) )
        
        return ( []  )

In [ ]:
for raw_record in train_dataset.take(10):
  print(repr(raw_record))

In [ ]:
train_dataset = train_dataset.map( read_stack_image, num_parallel_calls= AUTO )

In [ ]:
from keras.preprocessing.image import ImageDataGenerator



train_datagen = ImageDataGenerator()

In [ ]:
train_datagen.flow_from_directory( "../input/seti-breakthrough-listen/train/",  )

In [ ]:
from tf.keras.

In [ ]:
class DataGenerator ( utils.Sequence) :
    
    def __init__ (self, data_list, batch_size, #required_channels = [1, 3, 5 ] , 
                  image_height = 224, image_width = 224, 
                  shuffle = True,num_channel = 3  ):
        
        self.data_list = data_list
        self.batch_size = batch_size
        #self.channels_required = required_channels
        self.image_width = image_width
        self.image_height = image_height
        self.shuffle = True 
        self.num_channel =num_channel 
        self.is_there_left_over = False # this to check is there any left images after creating batch slices 
        
        self.number_of_batches = 0
        
        
    def on_epoch_end(self):
        self.indices = np.arrange( len(self.data_list ))
        
        if self.shuffle :
            np.random.shuffle( self.indices )
    
    def __len__(self ):
        
        number_of_batch =int ( len( self.data_list)/ self.batch_size )
        
        ''' below if loop to check for remaining data after slicing with respectto batch'''
        if number_of_batch * self.batch_size !=len( self.data_list):
            number_of_batch = number_of_batch +1
            self.is_there_left_over = True
        
        self.number_of_batches = number_of_batch
        
        return ( number_of_batch  )
    
    def __getitem__(self, idx = 0  ):
        
        x = np.empty( ( self.batch_size, self.image, self.num_channel ) )
        
        if self.is_there_left_over & (idx == (self.number_of_batches - 1 ) ) :
            
            indices = self.indices [ idx * self.batch_size :  ]
            
        else:
            
            indices = self.indices [ idx * self.batch_size : ( idx +1 )* self.batch_size ]
        
        for i, path in enumerate ( indices):
            x[i] = self.read_image( path )
        
        return x 
            
    def read_image ( self, path ):
        
        file_name=file_name_result
        data = np.load( path ).astype( np.float32 )    
        data = np.dstack( ( data[0], data[2], data[4] ) )
        
        return ( data )